In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import eppy

#iddfile = "/Applications/EnergyPlus-22-1-0/PreProcess/IDFVersionUpdater/V22-1-0-Energy+.idd"
iddfile = "/Users/sararhiger/Desktop/V9-6-0-Energy+.idd"
IDF.setiddname(iddfile)
idfname = "/Users/sararhiger/Desktop/Endelig_optimering.idf"
epwfile = "/Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw"

# idf1 = IDF(idfname)

#idf.run(output_directory="Resultater",ep_version="9-6-0") 


### Funktion der tjekker overophedningstimerne i hverdage fra 8-17 og sommerferien (28/6-8/8)

In [2]:
import numpy as np
from datetime import datetime

def check_max_temp(data, max_temp, start_tid, slut_tid, årstal):
    værelser = np.array([0]*84)

    for row in data['Rows']:
        key, value = list(row.items())[0]
        hour = int(key[6:8])
        month = int(key[0:2])
        day = int(key[3:5])

        if month==7 or (month==6 and day>=28) or (month==8 and day<=8):
            pass
        
        elif start_tid <= hour <= slut_tid:
            dato = datetime.strptime(key, '%m/%d %H:%M:%S').replace(year=årstal)
            if dato.weekday() < 5:  # 5 og 6 weekday er weekender
                # np.array(value) > max_temp er True eller False værdier (1 eller 0)
                værelser += np.array(value) > max_temp
                
    return værelser

### Træk 0.05 eller 0.001 fra flowrate indtil overophedningstimer overstiger 100 timer

In [3]:
import json
import numpy as np
idf = IDF(idfname, epwfile)
idf1 = IDF(idfname)
#idf.run(ep_version='22-1-0') 
idf.run(ep_version="9-6-0")

with open("eplusout_hourly.json") as f:
    data = json.load(f)

overtemp = check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)
print(overtemp)

for _ in range(30):
    for i in range(84):
        zone = idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]
        
        if overtemp[i] > 100 or zone["Outdoor_Air_Flow_Rate_per_Zone"] <= 0.001:
            pass

        else:
            if zone["Outdoor_Air_Flow_Rate_per_Zone"] < 0.01:
                zone["Outdoor_Air_Flow_Rate_per_Zone"] -= 0.001

            elif zone["Outdoor_Air_Flow_Rate_per_Zone"] < 0.1:
                zone["Outdoor_Air_Flow_Rate_per_Zone"] -= 0.01   

            elif zone["Outdoor_Air_Flow_Rate_per_Zone"] >= 0.1:
                zone["Outdoor_Air_Flow_Rate_per_Zone"] -= 0.05

    idf1.save()

    idfname = "/Users/sararhiger/Desktop/Endelig_optimering.idf"
    idf = IDF(idfname, epwfile)
    idf1 = IDF(idfname)
    idf.run(ep_version="9-6-0")
    #idf.run(ep_version='22-1-0') 

    with open("eplusout_hourly.json") as f:
        data = json.load(f)

    overtemp = check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)
    print(overtemp)




/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[ 77   0  41  31   0   0   3 188 181  19  22 154 103 105   6  11   0  20
  12  20 120 142   2   2  42  23   0   0   3   4   4   0   3   0   0  77
  21  20  16  13   0   3   5   1   3  12   0   0   0   0   0   0   0   0
   0   0   0   0   0  36 606 353 342 343 339 523  58  21  15  19   0   0
   0   0   0   0   0   0  15   0 140  65 114  38]

/Applications/EnergyPlus-9-6-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Users/sararhiger/Desktop/V9-6-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[145   5 129  98  49  49  56

KeyboardInterrupt: 

In [3]:
import json
import numpy as np 

idf = IDF(idfname, epwfile)
idf.run(ep_version='22-1-0') 

f=open("eplusout_hourly.json")
data = json.load(f)

overtemp=check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)
print(overtemp)

for _ in range(50):
    for i in range(84):
        zone = idf.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]

        if zone["Outdoor_Air_Flow_Rate_per_Zone"] >= 0.1 and overtemp[i] < 100:
            zone["Outdoor_Air_Flow_Rate_per_Zone"] -= 0.05
        
        elif zone["Outdoor_Air_Flow_Rate_per_Zone"] < 0.1 and zone["Outdoor_Air_Flow_Rate_per_Zone"] > 0.001 and overtemp[i] < 100:
            zone["Outdoor_Air_Flow_Rate_per_Zone"] -= 0.001
        
        else:
            pass
                    
    idf.save()

    idf = IDF(idfname, epwfile)
    
    idf.run(ep_version='22-1-0') 

    f=open("eplusout_hourly.json")
    data = json.load(f)

    overtemp=check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)

    print(overtemp)
    break



/Applications/EnergyPlus-22-1-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Applications/EnergyPlus-22-1-0/PreProcess/IDFVersionUpdater/V22-1-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[63 79  9 54 99 96 98 98 89 76 64 65  6  0  0  0  0 66 47 49 13 13  0 89
 10  8  8 10 65  0  0  0  7 59  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  6 78 50  7  2 11  4  0  0]

/Applications/EnergyPlus-22-1-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Applications/EnergyPlus-22-1-0/PreProcess/IDFVersionUpdater/V22-1-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[63 79  9 54 99 96 98 98 89 76 64 65  6

In [ ]:
idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][0]["Outdoor_Air_Flow_Rate_per_Zone"]

90

### Læg 0.001/0.002 til flowrate til overophedningstimerne lige præcis er under 100 eller 25 timer (26/27 grader)

In [25]:
import json
import numpy as np 

idf = IDF(idfname, epwfile)
idf1 = IDF(idfname)
idf.run(ep_version='22-1-0') 

f=open("eplusout_hourly.json")
data = json.load(f)

overtemp=check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)

for _ in range(60):
    for i in range(84):
        zone = idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem'][i]
        
        if overtemp[i] >= 100:
            zone["Outdoor_Air_Flow_Rate_per_Zone"] += 0.001
                   
    idf1.save()

    idfname = "/Users/sararhiger/Desktop/ENDELIG.idf"
    idf = IDF(idfname, epwfile)
    idf1 = IDF(idfname)
    
    idf.run(ep_version='22-1-0') 

    f=open("eplusout_hourly.json")
    data = json.load(f)

    overtemp=check_max_temp(data, max_temp=26, start_tid=8, slut_tid=17, årstal=2010)

    print(overtemp)


/Applications/EnergyPlus-22-1-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Applications/EnergyPlus-22-1-0/PreProcess/IDFVersionUpdater/V22-1-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf


/Applications/EnergyPlus-22-1-0/energyplus --weather /Users/sararhiger/Desktop/Bachelor-project/DRY_2001-2010_epw_fikset.epw --output-directory /Users/sararhiger/Desktop/Bachelor-project/Kode --idd /Applications/EnergyPlus-22-1-0/PreProcess/IDFVersionUpdater/V22-1-0-Energy+.idd --expandobjects /Users/sararhiger/Desktop/Bachelor-project/Kode/in.idf

[228 150 179 148 132 253 264 264 253 102  99 149 107  30  28  27  27  79
 108 192 108 112 129 299 240 217 227 258 237  30  30  30 174 213 183 220
 216  49  35  42 113  93  88  28  87 100 111  88 182 279 223 210  25  35
  63  80  52  41  35  23  27  31  31  27  30  35  68  56 177  91 101 147
 109

In [6]:
Final_flow=[round(x["Outdoor_Air_Flow_Rate_per_Zone"], 4) for x in idf1.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem']]

print(overtemp)

index=np.where(overtemp > 25)[0]

for i in index:
    print(overtemp[i], Final_flow[i], data['Cols'][i])

[150 121 155 109 105 246 268 269 262 106 109 136  87  40  31  31  32 175
 331 457 107 106  51 194 185 158 197 213 175  31  31  31 115 171 158 204
 196  64  46  58 103  90 107  41  58  34 137 104  99 200 198 185  36  31
  39  41  41  39  31  30  31  37  38  39  40  44  41  47 149  71  66 114
  95 113 150 118 111 145 188 200 135 279   0   0]
150 0.063 {'Units': 'C', 'Variable': '3075321:Zone Operative Temperature'}
121 0.017 {'Units': 'C', 'Variable': '3075325:Zone Operative Temperature'}
155 0.001 {'Units': 'C', 'Variable': '3075328:Zone Operative Temperature'}
109 0.001 {'Units': 'C', 'Variable': '3075333:Zone Operative Temperature'}
105 0.001 {'Units': 'C', 'Variable': '3075338:Zone Operative Temperature'}
246 0.001 {'Units': 'C', 'Variable': '3075341:Zone Operative Temperature'}
268 0.001 {'Units': 'C', 'Variable': '3075344:Zone Operative Temperature'}
269 0.079 {'Units': 'C', 'Variable': '3075347:Zone Operative Temperature'}
262 0.049 {'Units': 'C', 'Variable': '3075350:Zone Operati